# Code Matlab

In [ ]:
clear;
clc;
format long e    % çà ce n'est pas intelligent, la constante d'Euler c'est e dans Matlab: change de nom!

L = 1.0;  % longueur du lit
ug = 0.01; % vitesse spécifique de gaz
epsb = 0.5; % porosité du lit
kg = 0.0001; % coefficient de transfert
Ke = 10;  % coeff d'équilibre
dp = 0.005;   % diamètre de particule
as = 6*(1-epsb)/dp;  % surface spécifique, dépend du diamètre de la sphère
tend = 1000;  % fin du temps de simulation

% il n'y a pas de dispersion axiale

N = 100;
M = 5;
x = linspace(0, L, N+1);   % sépare le tube en N+1 compartiments de même longueur
t = linspace(0, tend, M+1);  % sépare le temps en 6 moments de même durée; çà me semble bien court pour une dynamique...

figure, hold on
title('Analytical solution:')
xlabel('x/L [-]');
h = legend('show','location','best');
set(h,'FontSize',12);
for j=2:M+1
    for i=1:N+1
        tau = kg*as/((1-epsb)*Ke)*(t(j) - x(i)/ug);  % donne le \tau de la correction pour chaque x et L
        xi = kg*as/(epsb*ug)*x(i);  % pareil avec le \xi
        fnc = @(u) exp(-u).*besseli(0, sqrt(4*tau*u), 1)./exp(-abs(real(sqrt(4*tau*u))));  % scales besseli function to avoid overflow
        I = integral(fnc, 0 , xi); % integral in the solution
        e = exp(-tau); % no comment
        g = besseli(0, sqrt(4*tau*xi))*exp(-xi); % la deuxième partie dans le Cs
        Cg(i) = real(1 - e*I);   % donne le Cg, normalisé par Cg_in
        Cs(i) = real(Ke*(1 - e*(I + g))); % pareil pour Cs
    end
    Cg(isinf(Cg)) = 0;
    Cs(isinf(Cs)) = 0;

    subplot(2,1,1), hold on
    yyaxis left, hold on
    plot(x/L,Cg,'-x','LineWidth',1,'MarkerSize',1,'DisplayName',num2str(t(j)))
    ylabel('Cg/Cgin [-]');
    ylim([0 1]);

    yyaxis right, hold on
    subplot(2,1,2), hold on
    plot(x/L,Cs,'-x','LineWidth',1,'MarkerSize',1,'DisplayName',num2str(t(j)))
    ylabel('Cs/Cgin [-]');
    ylim([0 Ke]);
end

# Attempt - PINN in python

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
import numpy as np
import deepxde as dde
from deepxde.backend import tf
import matplotlib.pyplot as plt

In [ ]:
L = 1.0;            # longueur du lit
ug = 0.01;          # vitesse spécifique de gaz
epsb = 0.5;         # porosité du lit
kg = 0.0001;        # coefficient de transfert
Ke = 10;            # coeff d'équilibre
dp = 0.005;         # diamètre de particule
as = 6*(1-epsb)/dp; # surface spécifique, dépend du diamètre de la sphère
tend = 1000;        # fin du temps de simulation

Spécification des dimensions (z,t)

In [ ]:
# Pas de calcul (0.1) ?
geom = dde.geometry.Interval(0, L)
timedomain = dde.geometry.TimeDomain(0, 0.99)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

Partial Differential Equations

In [ ]:
def pde(z, cg):
    dcg_z = dde.grad.jacobian(cg, z, i=0, j=0)
    dcg_t = dde.grad.jacobian(cg, z, i=0, j=1)
    return (dcg_t + ug * dcg_z) + (kg*as/epsb) * (cg-cs/Ke)

def pde2(z, cs):
    dcs_t = dde.grad.jacobian(y, x, i=0, j=1)
    return dcs_t - (kg*as/(1-epsb)) * (cg-cs/Ke)

Initial Condition & Boundary Condition

In [ ]:
bc = dde.icbc.DirichletBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(geomtime, lambda x: -np.sin(np.pi * x[:, 0:1]), lambda _, on_initial: on_initial)